In [121]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [122]:
from datetime import datetime

In [123]:
results = []

month_map = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09',
             'Oct':'10','Nov':'11','Dec':'12'}

data_history = {
    "Date": [],
    "source": [],
    "sentiment":[]
}
#stocknewsapi has LLM -GPT4 analyzed sentiment --> which categorizes the news into the buckets positive, negative, neutral
for i in range (1,100):#max10000 data 
    api_url = "https://stocknewsapi.com/api/v1?tickers=AMZN&data=yeartodate&items=100&page="+ str(i) +"&token=metobjbevvotlrizynavabhyisukldmxcxsdwgrf"
    response = requests.get(api_url)
    response_data = response.json()
    for rd in response_data["data"]:
        date_text = rd["date"].split(' ')
        dt = date_text[3] + month_map[date_text[2]] + date_text[1]
        data_history["Date"].append(pd.Timestamp(dt))
        data_history["source"].append(rd["source_name"])
        data_history["sentiment"].append(rd["sentiment"])
    results += response_data["data"]
import pandas as pd
df = pd.DataFrame(data_history)
print(df)

           Date           source sentiment
0    2023-08-08          Reuters  Positive
1    2023-08-08    InvestorPlace  Positive
2    2023-08-08    New York Post  Positive
3    2023-08-08    InvestorPlace  Positive
4    2023-08-08    Seeking Alpha   Neutral
...         ...              ...       ...
9895 2021-07-20             CNBC  Negative
9896 2021-07-20     Market Watch  Negative
9897 2021-07-20       TechXplore   Neutral
9898 2021-07-20    Seeking Alpha   Neutral
9899 2021-07-20  The Motley Fool   Neutral

[9900 rows x 3 columns]


In [124]:
options_data = pd.read_csv('/Users/manaswiniswamy/Desktop/Book1-Options.csv',parse_dates=True,index_col='Date')
options_data

,call_volume,call_percentage,put_volume,put_percentage
Date,,,,
2022-08-05,1893592,0.57,1401144,0.43
2022-08-08,1339990,0.58,978214,0.42
2022-08-09,901492,0.54,761950,0.46
2022-08-10,605572,0.52,558114,0.48
2022-08-11,843904,0.52,763680,0.48
...,...,...,...,...
2023-08-01,491712,0.61,315620,0.39
2023-08-02,472038,0.63,277290,0.37
2023-08-03,749284,0.62,450228,0.38


In [125]:
# fetch the derivatives data (this decides the volatility and the vix index)--> will integrate it today

In [126]:
df['sentiment'] = df['sentiment'].map({'Positive': 1, 'Negative': -1, 'Neutral':0})
df

,Date,source,sentiment
0,2023-08-08,Reuters,1
1,2023-08-08,InvestorPlace,1
2,2023-08-08,New York Post,1
3,2023-08-08,InvestorPlace,1
4,2023-08-08,Seeking Alpha,0
...,...,...,...
9895,2021-07-20,CNBC,-1
9896,2021-07-20,Market Watch,-1
9897,2021-07-20,TechXplore,0
9898,2021-07-20,Seeking Alpha,0


In [127]:
stock_data = pd.read_csv('AMZN.csv', parse_dates=True, index_col='Date')

In [ ]:
dsd_anaysis = pd.read_csv('demandSupply.csv',parse_dates=True, index_col='Date')

In [128]:
stock_data['200_day_avg'] = stock_data['Close'].rolling(window=200).mean()
stock_data['50_day_avg'] = stock_data['Close'].rolling(window=50).mean()
stock_data['10_day_avg'] = stock_data['Close'].rolling(window=10).mean()

In [129]:
stock_data.dropna(inplace=True)

In [130]:
stock_data['Trend'] = np.where(stock_data['Close'].shift(-1) > stock_data['Close'], 1, 0)

In [131]:
stock_data=stock_data.merge(df,how="inner",on="Date")
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,200_day_avg,50_day_avg,10_day_avg,Trend,source,sentiment
0,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,InvestorPlace,1
1,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0
2,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0
3,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,CNN Business,-1
4,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Zacks Investment Research,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Market Watch,0
969,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,WSJ,0
970,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,TechXplore,1
971,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Proactive Investors,1


In [132]:
stock_data=stock_data.merge(options_data,how="inner",on="Date")
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,200_day_avg,50_day_avg,10_day_avg,Trend,source,sentiment,call_volume,call_percentage,put_volume,put_percentage
0,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,InvestorPlace,1,724532,0.58,516656,0.42
1,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0,724532,0.58,516656,0.42
2,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0,724532,0.58,516656,0.42
3,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,CNN Business,-1,724532,0.58,516656,0.42
4,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Zacks Investment Research,0,724532,0.58,516656,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Market Watch,0,1656752,0.61,1081218,0.39
969,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,WSJ,0,1656752,0.61,1081218,0.39
970,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,TechXplore,1,1656752,0.61,1081218,0.39
971,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Proactive Investors,1,1656752,0.61,1081218,0.39


In [ ]:
stock_data=stock_data.merge(dsd_anaysis,how="inner",on="Date")
stock_data

In [133]:
X = stock_data[['200_day_avg', '50_day_avg', '10_day_avg','sentiment','call_volume','call_percentage','put_volume','put_percentage']].values
y = stock_data['Trend'].values

In [134]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [136]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [137]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [138]:
model.fit(X_train, y_train, epochs=2000, batch_size=32, validation_split=0.1)

Epoch 1/2000
22/22 [==============================] - 0s 5ms/step - loss: 0.6756 - accuracy: 0.5814 - val_loss: 0.6804 - val_accuracy: 0.6282
Epoch 2/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.6603 - accuracy: 0.6043 - val_loss: 0.6680 - val_accuracy: 0.6282
Epoch 3/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.6497 - accuracy: 0.6143 - val_loss: 0.6553 - val_accuracy: 0.6538
Epoch 4/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.6365 - accuracy: 0.6371 - val_loss: 0.6411 - val_accuracy: 0.7051
Epoch 5/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.6224 - accuracy: 0.7057 - val_loss: 0.6284 - val_accuracy: 0.7179
Epoch 6/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.6065 - accuracy: 0.7057 - val_loss: 0.6098 - val_accuracy: 0.7436
Epoch 7/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.5872 - accuracy: 0.7186 - val_loss: 0.6059 - val_accuracy: 0.7179
Epoch 

22/22 [==============================] - 0s 1ms/step - loss: 0.2689 - accuracy: 0.8886 - val_loss: 0.3506 - val_accuracy: 0.8718
Epoch 59/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2670 - accuracy: 0.8843 - val_loss: 0.3363 - val_accuracy: 0.8718
Epoch 60/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2632 - accuracy: 0.8857 - val_loss: 0.3281 - val_accuracy: 0.8718
Epoch 61/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2613 - accuracy: 0.8786 - val_loss: 0.3391 - val_accuracy: 0.8718
Epoch 62/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2666 - accuracy: 0.8829 - val_loss: 0.3379 - val_accuracy: 0.8718
Epoch 63/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2674 - accuracy: 0.8914 - val_loss: 0.3122 - val_accuracy: 0.8590
Epoch 64/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2674 - accuracy: 0.8986 - val_loss: 0.3522 - val_accuracy: 0.8718
Epoch 65/2000

22/22 [==============================] - 0s 1ms/step - loss: 0.1544 - accuracy: 0.9571 - val_loss: 0.2573 - val_accuracy: 0.8846
Epoch 116/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1631 - accuracy: 0.9486 - val_loss: 0.2416 - val_accuracy: 0.8846
Epoch 117/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1563 - accuracy: 0.9500 - val_loss: 0.2571 - val_accuracy: 0.8846
Epoch 118/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1579 - accuracy: 0.9500 - val_loss: 0.2544 - val_accuracy: 0.8846
Epoch 119/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1579 - accuracy: 0.9543 - val_loss: 0.2073 - val_accuracy: 0.8590
Epoch 120/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1580 - accuracy: 0.9471 - val_loss: 0.2308 - val_accuracy: 0.8846
Epoch 121/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1623 - accuracy: 0.9443 - val_loss: 0.2288 - val_accuracy: 0.8846
Epoch 1

Epoch 172/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1094 - accuracy: 0.9671 - val_loss: 0.1731 - val_accuracy: 0.8846
Epoch 173/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1102 - accuracy: 0.9643 - val_loss: 0.2012 - val_accuracy: 0.8974
Epoch 174/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1087 - accuracy: 0.9686 - val_loss: 0.1739 - val_accuracy: 0.8846
Epoch 175/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1038 - accuracy: 0.9671 - val_loss: 0.1560 - val_accuracy: 0.9359
Epoch 176/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1041 - accuracy: 0.9643 - val_loss: 0.1580 - val_accuracy: 0.8974
Epoch 177/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1066 - accuracy: 0.9729 - val_loss: 0.1677 - val_accuracy: 0.8846
Epoch 178/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1118 - accuracy: 0.9671 - val_loss: 0.1531 - val_accuracy:

Epoch 229/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.0788 - accuracy: 0.9743 - val_loss: 0.1296 - val_accuracy: 0.9359
Epoch 230/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.0769 - accuracy: 0.9771 - val_loss: 0.1203 - val_accuracy: 0.9359
Epoch 231/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.0727 - accuracy: 0.9843 - val_loss: 0.1219 - val_accuracy: 0.9487
Epoch 232/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.0721 - accuracy: 0.9829 - val_loss: 0.1160 - val_accuracy: 0.9487
Epoch 233/2000
22/22 [==============================] - 0s 2ms/step - loss: 0.0756 - accuracy: 0.9829 - val_loss: 0.1461 - val_accuracy: 0.8974
Epoch 234/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.0725 - accuracy: 0.9871 - val_loss: 0.1170 - val_accuracy: 0.9744
Epoch 235/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.0703 - accuracy: 0.9829 - val_loss: 0.1244 - val_accuracy:

...

22/22 [==============================] - 0s 1ms/step - loss: 1.5833e-04 - accuracy: 1.0000 - val_loss: 2.1989e-04 - val_accuracy: 1.0000
Epoch 1768/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.5067e-04 - accuracy: 1.0000 - val_loss: 2.2288e-04 - val_accuracy: 1.0000
Epoch 1769/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.4482e-04 - accuracy: 1.0000 - val_loss: 2.2112e-04 - val_accuracy: 1.0000
Epoch 1770/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.4665e-04 - accuracy: 1.0000 - val_loss: 2.1752e-04 - val_accuracy: 1.0000
Epoch 1771/2000
22/22 [==============================] - 0s 2ms/step - loss: 1.4460e-04 - accuracy: 1.0000 - val_loss: 2.1494e-04 - val_accuracy: 1.0000
Epoch 1772/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.4554e-04 - accuracy: 1.0000 - val_loss: 2.2231e-04 - val_accuracy: 1.0000
Epoch 1773/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.4716e-04 - accuracy

22/22 [==============================] - 0s 2ms/step - loss: 1.1891e-04 - accuracy: 1.0000 - val_loss: 1.6585e-04 - val_accuracy: 1.0000
Epoch 1821/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.2040e-04 - accuracy: 1.0000 - val_loss: 1.8378e-04 - val_accuracy: 1.0000
Epoch 1822/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.2186e-04 - accuracy: 1.0000 - val_loss: 1.6373e-04 - val_accuracy: 1.0000
Epoch 1823/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.1879e-04 - accuracy: 1.0000 - val_loss: 1.6419e-04 - val_accuracy: 1.0000
Epoch 1824/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.2757e-04 - accuracy: 1.0000 - val_loss: 1.5881e-04 - val_accuracy: 1.0000
Epoch 1825/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.1837e-04 - accuracy: 1.0000 - val_loss: 1.6864e-04 - val_accuracy: 1.0000
Epoch 1826/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.1567e-04 - accuracy

22/22 [==============================] - 0s 1ms/step - loss: 1.0354e-04 - accuracy: 1.0000 - val_loss: 1.3244e-04 - val_accuracy: 1.0000
Epoch 1874/2000
22/22 [==============================] - 0s 1ms/step - loss: 9.8875e-05 - accuracy: 1.0000 - val_loss: 1.2575e-04 - val_accuracy: 1.0000
Epoch 1875/2000
22/22 [==============================] - 0s 1ms/step - loss: 9.6154e-05 - accuracy: 1.0000 - val_loss: 1.4137e-04 - val_accuracy: 1.0000
Epoch 1876/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.0979e-04 - accuracy: 1.0000 - val_loss: 1.2408e-04 - val_accuracy: 1.0000
Epoch 1877/2000
22/22 [==============================] - 0s 1ms/step - loss: 1.0099e-04 - accuracy: 1.0000 - val_loss: 1.2458e-04 - val_accuracy: 1.0000
Epoch 1878/2000
22/22 [==============================] - 0s 1ms/step - loss: 9.5024e-05 - accuracy: 1.0000 - val_loss: 1.2568e-04 - val_accuracy: 1.0000
Epoch 1879/2000
22/22 [==============================] - 0s 1ms/step - loss: 9.4550e-05 - accuracy

22/22 [==============================] - 0s 1ms/step - loss: 6.7683e-05 - accuracy: 1.0000 - val_loss: 9.1285e-05 - val_accuracy: 1.0000
Epoch 1927/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.9868e-05 - accuracy: 1.0000 - val_loss: 9.3403e-05 - val_accuracy: 1.0000
Epoch 1928/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.6835e-05 - accuracy: 1.0000 - val_loss: 9.5568e-05 - val_accuracy: 1.0000
Epoch 1929/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.8597e-05 - accuracy: 1.0000 - val_loss: 1.0927e-04 - val_accuracy: 1.0000
Epoch 1930/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.6595e-05 - accuracy: 1.0000 - val_loss: 9.6716e-05 - val_accuracy: 1.0000
Epoch 1931/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.8842e-05 - accuracy: 1.0000 - val_loss: 9.5473e-05 - val_accuracy: 1.0000
Epoch 1932/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.7894e-05 - accuracy

22/22 [==============================] - 0s 1ms/step - loss: 5.2810e-05 - accuracy: 1.0000 - val_loss: 7.0659e-05 - val_accuracy: 1.0000
Epoch 1980/2000
22/22 [==============================] - 0s 1ms/step - loss: 5.2674e-05 - accuracy: 1.0000 - val_loss: 7.4347e-05 - val_accuracy: 1.0000
Epoch 1981/2000
22/22 [==============================] - 0s 1ms/step - loss: 5.1041e-05 - accuracy: 1.0000 - val_loss: 6.7992e-05 - val_accuracy: 1.0000
Epoch 1982/2000
22/22 [==============================] - 0s 1ms/step - loss: 4.9499e-05 - accuracy: 1.0000 - val_loss: 7.7053e-05 - val_accuracy: 1.0000
Epoch 1983/2000
22/22 [==============================] - 0s 1ms/step - loss: 5.1271e-05 - accuracy: 1.0000 - val_loss: 6.5544e-05 - val_accuracy: 1.0000
Epoch 1984/2000
22/22 [==============================] - 0s 1ms/step - loss: 5.2275e-05 - accuracy: 1.0000 - val_loss: 6.7297e-05 - val_accuracy: 1.0000
Epoch 1985/2000
22/22 [==============================] - 0s 1ms/step - loss: 5.4181e-05 - accuracy

In [139]:
loss, accuracy = model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 878us/step - loss: 0.0261 - accuracy: 0.9949


In [140]:
print(f"Test accuracy: {accuracy}")

Test accuracy: 0.9948717951774597
